# Tutorial: Analyse data downloaded from Google Earth Engine using pyveg 


Google Earth Engine is a powerful tool for obtaining and analysing satellite imagery. The pyveg package provides useful scripts for interacting with the Earth Engine API and downloading data. 

The location used in this tutorial is Tiger Bush vegetation from Niger in coordinates 2.59, 13.12. The downloaded data is a JSON file containing weather and network centrality metrics in a monthly basis from 2015 to 2020.

Now lets use the functions provided by pyveg to run a simple analysis on the data.

In [1]:
import argparse
import os
from matplotlib import pyplot as plt

from pyveg.src.data_analysis_utils import *
from pyveg.src.plotting import *
from pyveg.src.image_utils import create_gif_from_images

Input dataset is a json file found in this directory. 

In [2]:
# results directory from `download_gee_data` script.
json_summary_path =  'results_summary_TigerBush_Niger.json'

The output figures will be saved in an `analysis` sub-directory. 

In [3]:
# put output plots in the results dir
input_dir = '.'
output_dir = os.path.join(input_dir, 'analysis')

Read all json files in the directory and produce a dictionary of dataframes. Each key is a satellite, either weather related or image related (for the network centrality measures).


In [4]:
print(f"Reading results from '{os.path.abspath(json_summary_path)}'...")
dfs = variable_read_json_to_dataframe(json_summary_path)

print (dfs.keys())

Reading results from '/Users/crangelsmith/PycharmProjects/monitoring-ecosystem-resilience/notebooks/tutorials/results_summary_TigerBush_Niger.json'...
dict_keys(['COPERNICUS/S2', 'ECMWF/ERA5/MONTHLY'])


This is how the output dataframe look:

In [5]:
print (dfs['COPERNICUS/S2'].head())
print (dfs['ECMWF/ERA5/MONTHLY'].head())

         date                                        feature_vec  latitude  \
0  2015-07-15  [0.0, -31.0, -59.0, -71.0, -116.0, -130.0, -22...    2.6377   
1  2015-07-15  [0.0, -17.0, -69.0, -104.0, -169.0, -200.0, -2...    2.5514   
2  2015-07-15  [0.0, -49.0, -103.0, -171.0, -249.0, -292.0, -...    2.5741   
3  2015-07-15  [0.0, -13.0, -39.0, -62.0, -103.0, -132.0, -20...    2.5923   
4  2015-07-15  [0.0, -33.0, -81.0, -112.0, -164.0, -226.0, -2...    2.6286   

   longitude    mean  offset  offset50  slope         std  
0    13.1086 -456.70 -1082.0    -640.0 -54.10  343.613897  
1    13.1223 -459.10 -1026.0    -568.0 -51.30  318.552492  
2    13.1405 -621.65 -1325.0    -684.0 -66.25  406.338440  
3    13.1359 -398.95  -919.0    -527.0 -45.95  292.988818  
4    13.1495 -516.35 -1145.0    -617.0 -57.25  357.583176  
         date  mean_2m_air_temperature  total_precipitation
0  2015-01-16               296.405121             0.000000
1  2015-02-15               303.000031             

## Spatial analysis

First, lets build 2D plots showing the network centrality values on the general 10km images for each date. 

In [ ]:
# spatial analysis and plotting 
# from the dataframe, produce network metric figure for each avalaible date
print('\nCreating spatial plots...')

# create new subdir for time series analysis
spatial_subdir = os.path.join(output_dir, 'spatial')
if not os.path.exists(spatial_subdir):
    os.makedirs(spatial_subdir, exist_ok=True)

for collection_name, df in dfs.items():
    if collection_name == 'COPERNICUS/S2' or 'LANDSAT' in collection_name:
        # convert the dataframe of each image to geopandas and coarse its resolution slightly
        data_df_geo = convert_to_geopandas(df.copy())
        data_df_geo_coarse = coarse_dataframe(data_df_geo, 4)
        create_lat_long_metric_figures(data_df_geo_coarse, 'offset50', spatial_subdir)

output_plots_name = create_gif_from_images(spatial_subdir,'output.gif')


Creating spatial plots...


Lets visualise the result on a GIF.

In [ ]:
from IPython.display import Image
with open(name,'rb') as f:
    display(Image(data=f.read(), format='png',width=500, height=500))

The average network centrality feature vectors ver all time points and sub images are the following:

In [ ]:
# create new subdir for time series analysis
tsa_subdir = output_dir

 # remove outliers from the time series
dfs = drop_veg_outliers(dfs, sigmas=3) # not convinced this is really helping much

# plot the feature vectors averaged over all time points and sub images
try:
    plot_feature_vectors(dfs, tsa_subdir)
except AttributeError:
    print('Can not plot feature vectors...') 
            


## Time series analysis

Using the data we can build a time series. For this analysis we do the following steps:

- Build time series for every sub-image, we drop points with large outliers and smooth the sub-image time series.
- We average all the network centrality measures from every sub-image into a single time series.
- Compare time series with precipitation data and calculate measures such a correlation, AR1, Kendal tau, etc.

In [ ]:
# convert dataframe to time series
time_series_dfs = make_time_series(dfs.copy())

# LOESS smoothing on sub-image time series
smoothed_time_series_dfs = make_time_series(smooth_veg_data(dfs.copy(), n=4)) # increase smoothing with n>5

# make a smoothed time series plot
plot_smoothed_time_series(smoothed_time_series_dfs, tsa_subdir)

Explore auto-correlation between both smoothed and un-smoothed time series.

In [ ]:
# make autocorrelation plots
plot_autocorrelation_function(smoothed_time_series_dfs, tsa_subdir)


Investigate the cross-correlation between the network centrality measures and precipitation for different lags of time.

In [ ]:
# make cross correlation scatterplot matrix plots
plot_cross_correlations(smoothed_time_series_dfs, tsa_subdir)


## Seasonal and trend analysis

The time series shown above show a clear seasonal trend. The STL decomposition implementation from the statsmodels package is applied to the un-smoothed time series to separate the different components. 

This is done for both the network centrality metrics and precipitation data.

In [ ]:
do_stl_decomposition(time_series_dfs,12,tsa_subdir)


The time series is detrended by a first order differentiation (substracting the values from the last month in the 12 period). 

The same time series analysis (AR1, correlation, Kendal Tau)is then done on the de-trended time series.

In [ ]:
#   remove seasonality in the summary time series
time_series_uns_summary_dfs = remove_seasonality_combined(smoothed_time_series_dfs.copy(), 12, "M")

# make a smoothed time series plot
plot_smoothed_time_series(time_series_uns_summary_dfs, tsa_subdir, '-no-seasonality-summary-ts',plot_std = False)
